In [1]:
import networkx as nx
import plotly.graph_objects as go
import itertools
from collections import Counter
import pandas as pd

In [2]:
countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Cambodia", "Cameroon", "Canada", "Cape Verde", "Central African Republic", "Chad",
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cuba", "Cyprus",
    "Czech Republic", "North Korea", "Democratic Republic of the Congo", "Denmark", "Djibouti", "Dominica",
    "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini",
    "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada",
    "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hungary", "Iceland", "India",
    "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya",
    "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein",
    "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands",
    "Mauritania", "Mauritius", "Mexico", "Micronesia", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique",
    "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria",
    "North Macedonia", "Norway", "Oman", "Pakistan", "Palau", "Panama", "Papua New Guinea", "Paraguay", "Peru",
    "Philippines", "Poland", "Portugal", "Qatar", "South Korea", "Moldova", "Romania", "Russia", "Rwanda",
    "Saint Kitts and Nevis", "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino",
    "São Tomé and Príncipe", "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore",
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa", "South Sudan", "Spain", "Sri Lanka",
    "Sudan", "Suriname", "Sweden", "Switzerland", "Syria", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste",
    "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu", "Venezuela",
    "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

atlas_graph = nx.DiGraph()
atlas_graph.add_nodes_from(countries)
edges = [
    (c1, c2) for c1, c2 in itertools.product(countries, repeat=2)
    if c1 != c2 and c1.lower()[-1] == c2.lower()[0]
]
atlas_graph.add_edges_from(edges)

print(f"Graph created with {atlas_graph.number_of_nodes()} countries and {atlas_graph.number_of_edges()} possible moves.")

Graph created with 193 countries and 2014 possible moves.


In [3]:
def detect_and_compare_communities(G):
    print("COMMUNITY DETECTION ANALYSIS")

    # Community detection works best on undirected graphs
    undirected_G = G.to_undirected()

    #1: Louvain Method
    print("\nRunning 1: Louvain Method")
    louvain_communities = nx.community.louvain_communities(undirected_G, seed=42)
    louvain_modularity = nx.community.modularity(undirected_G, louvain_communities)
    print(f"Louvain found {len(louvain_communities)} communities with a modularity of {louvain_modularity:.4f}")

    #2: Girvan-Newman Method
    print("\nRunning 2: Girvan-Newman Method")
    gn_communities_generator = nx.community.girvan_newman(G)
    top_level_communities = next(gn_communities_generator)


    best_modularity = -1
    best_partition = None
    for partition in map(frozenset, top_level_communities):
        modularity = nx.community.modularity(undirected_G, [set(p) for p in top_level_communities])
        if modularity > best_modularity:
            best_modularity = modularity
            best_partition = [set(p) for p in top_level_communities]

    girvan_newman_communities = best_partition
    girvan_newman_modularity = best_modularity
    print(f"Girvan-Newman found {len(girvan_newman_communities)} communities with a modularity of {girvan_newman_modularity:.4f}")

    # Comparison
    print("Comparison: ")
    if louvain_modularity > girvan_newman_modularity:
        print("Best: Louvain Method has a higher modularity score.")
        best_communities = louvain_communities
        best_algorithm_name = "Louvain"
    else:
        print("Best: Girvan-Newman Method has a higher modularity score.")
        best_communities = girvan_newman_communities
        best_algorithm_name = "Girvan-Newman"

    community_map = {node: i for i, community in enumerate(best_communities) for node in community}

    return community_map, best_algorithm_name

In [4]:
def visualize_communities(G, community_map, algorithm_name):
    pos = nx.kamada_kawai_layout(G)

    node_x, node_y, node_hovertext, node_color, node_size = [], [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)

        node_x.append(x)
        node_y.append(y)

        hovertext = (
            f"<b>{node}</b><br>"
            f"Community ID: {community_map.get(node, 'N/A')}<br>"
            f"Options From Here: {out_degree}<br>"
            f"Paths To Here: {in_degree}"
        )
        node_hovertext.append(hovertext)
        node_color.append(community_map.get(node, -1))
        node_size.append(5 + (in_degree + out_degree) / 2)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_hovertext,
        marker=dict(
            showscale=False,
            colorscale='viridis',
            color=node_color,
            size=node_size,
            line_width=1,
            line_color='black'
        )
    )

    fig = go.Figure(data=[node_trace],
             layout=go.Layout(
                title=f'Atlas Game Communities ({algorithm_name} Method)',
                titlefont_size=20,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=5, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                plot_bgcolor='white',
                height=700,
                width=1000
            )
    )
    fig.show()

In [5]:
community_map, best_algo_name = detect_and_compare_communities(atlas_graph)

visualize_communities(atlas_graph, community_map, best_algo_name)

COMMUNITY DETECTION ANALYSIS

Running 1: Louvain Method
Louvain found 6 communities with a modularity of 0.4522

Running 2: Girvan-Newman Method
Girvan-Newman found 2 communities with a modularity of -0.0000
Comparison: 
Best: Louvain Method has a higher modularity score.
